# CICE crash investigation in ACCESS-OM2_01

Author: Andrew Kiss

In [9]:
%matplotlib inline

from glob import glob
import os,sys
import matplotlib.pyplot as plt
import numpy as np
import netCDF4 as nc
from tqdm import tqdm_notebook
from mpl_toolkits.basemap import Basemap
from calendar import month_abbr
from netCDF4 import Dataset
import cmocean as cm
import string

In [10]:
# figdir = 'figures/access-om2-01/Ice_Daily_ACCESS-OM2-01/'
figdir = '/g/data/v45/aek156/figures/access-om2-01/Ice_Crash_ACCESS-OM2-01/'
if not os.path.exists(figdir):
    os.makedirs (figdir)

def savefigure(fname):
#     plt.savefig(os.path.join(figdir, fname),dpi=100, bbox_inches="tight")  # comment out to disable saving
    plt.savefig(os.path.join(figdir, fname),dpi=50, bbox_inches="tight")  # comment out to disable saving
    return

## Investigate extreme winds in `/g/data1/ua8/JRA55-do/RYF/v1-3/RYF.u_10.1984_1985.nc`, `/g/data1/ua8/JRA55-do/RYF/v1-3/RYF.v_10.1984_1985.nc`

In [11]:
ncFile = nc.Dataset('/g/data1/ua8/JRA55-do/RYF/v1-3/RYF.u_10.1984_1985.nc', mode='r')
u10 = ncFile.variables['uas_10m'][...]

ncFile = nc.Dataset('/g/data1/ua8/JRA55-do/RYF/v1-3/RYF.v_10.1984_1985.nc', mode='r')
v10 = ncFile.variables['vas_10m'][...]


In [12]:
[np.min(u10), np.max(u10), np.min(v10), np.max(v10)]

[-45.070335, 40.71067, -43.399582, 40.831322]

These values all look reasonable. **So why do we have warnings of stress in excess of 10Pa in MOM?** Is it actually ice stress, not wind stress...?

In [13]:
# model data paths:
basepath = '/g/data3/hh5/tmp/cosima/'
model = 'access-om2-01'
expt = '01deg_jra55v13_ryf8485_spinup6'
# expt = '01deg_jra55v13_ryf8485_spinup6_dragio_0'
# expt = '01deg_jra55v13_ryf8485_spinup6_tripole_bug'

# model = 'access-om2-025'
# expt = '025deg_jra55v13_ryf8485_KDS75'

# model = 'access-om2'
# expt = '1deg_jra55_ryf8485_gfdl50'
# # expt = '1deg_jra55_ryf04_kds50'

# # Paul's mom01v5 MOM-SIS runs
# # /short/v45/pas561/mom/archive/kds75_wp2/output473
# # it will be moved to /g/data3/hh5/tmp/cosima/mom01v5/kds75_wp2/ at some point.
# model = 'mom01v5'
# expt = 'kds75_wp2'

DataDir = os.path.join(basepath, model)
expdir = os.path.join(DataDir, expt)
# date = '0004-07-05' # nicest view
# date = '0004-07-12'
# fn01 = '/g/data3/hh5/tmp/cosima/' + model + '/' +  expt + '/output023/ice/OUTPUT/iceh.' + date + '.nc'

In [14]:
# get model grid data:
gridFileList = glob(os.path.join(expdir, 'output*/ocean/ocean_grid.nc'))
# gridFileList = glob('/g/data3/hh5/tmp/cosima/access-om2-01/01deg_jra55v13_ryf8485_spinup6_dragio_0/output*/ocean/ocean_grid.nc')
# gridFileList = ['/short/v45/pas561/mom/archive/kds75_wp2/output473/ocean_grid.nc']
# # Paul's mom01v5 MOM-SIS runs
# gridFileList = ['/g/data3/hh5/tmp/cosima/mom01v5/kds75_wp2/output473/ocean_grid.nc']


# gridFileList = glob('/g/data3/hh5/tmp/cosima/access-om2-025/025deg_jra55v13_ryf8485_KDS75/output017/ocean/ocean_grid.nc')

gridFileList.sort()
ncFile = nc.Dataset(gridFileList[0], mode='r')
xt_ocean = ncFile.variables['xt_ocean'][...]
yt_ocean = ncFile.variables['yt_ocean'][...]
lon_t = ncFile.variables['geolon_t'][...]
lat_t = ncFile.variables['geolat_t'][...]
area_t = ncFile.variables['area_t'][...]
# NLAT_half = int(np.shape(area_t)[0]/2)
ht = ncFile.variables['ht'][...]
land_mask = np.copy(ht)
land_mask[np.where(ht>30)] = 0
land_mask[np.where(ht<=30)] = 1
ncFile.close()

In [19]:
icehFileList = glob(os.path.join(expdir, 'output*/ice/OUTPUT/iceh.0017-08*.nc'))
icehFileList = glob(os.path.join(expdir, 'output173/ice/OUTPUT/iceh.0017-08*.nc'))
icehFileList = glob(os.path.join(expdir, 'output*/ice/OUTPUT/iceh.0026-*-*.nc'))

icehFileList = glob(os.path.join(expdir, 'output*/ice/OUTPUT/iceh.0027-12-*.nc'))

icehFileList = glob(os.path.join(expdir, 'output*/ice/OUTPUT/iceh.0029-01-0*.nc'))

# icehFileList = glob(os.path.join(expdir, 'output*/ice/OUTPUT/iceh.0026-12-3*.nc'))

# icehFileList = glob(os.path.join(expdir, 'output*/ice/OUTPUT/iceh.0027-01-0*.nc'))





# icehFileList = glob(os.path.join(expdir, 'output173//ocean/ocean_month.nc'))


# icehFileList = glob(os.path.join(expdir, 'output*/ocean/o2i.nc'))

# icehFileList = glob(os.path.join(expdir, 'output*/ocean/ocean_month.nc'))

# icehFileList = glob(os.path.join(expdir, 'output*/ice/OUTPUT/iceh.????-07.nc'))
# icehFileList = glob('/g/data3/hh5/tmp/cosima/access-om2-025/025deg_jra55v13_ryf8485_KDS75/output*/ice/OUTPUT/iceh*07.nc')

icehFileList.sort()

In [ ]:
varnames = ['hi', 'aice', 'uvel', 'vvel', 'divu', 'shear', 'sig1', 'sig2', 
            'strairx', 'strairy', 'uocn', 'vocn', 'sst', 'sss', 'icespeed', 'oceanspeed',
            'strocnx', 'strocny']

varnames = ['hi', 'aice','uvel', 'vvel', 'shear', 'uocn', 'vocn', 'strairx', 'strairy', 'divu']

# varnames = ['uvel', 'vvel', 'strairx', 'strairy']

varnames = ['aice']

# varnames = ['uvel']

varnames = ['strocnx', 'strocny']

# varnames = ['shear','uocn','uvel']

# varnames = ['ssu_i', 'ssv_i'] # from ocean/o2i.nc

# varnames = ['tau_x',  'tau_y'] # from ocean/ocean_month.nc


# # for Paul's mom01v5 MOM-SIS runs
# icehFileList = glob(os.path.join(expdir, 'output473/ice__*.nc'))
# icehFileList.sort()
# varnames = ['UI', 'VI', 'SIGI', 'SIGII']
# # varnames = ['UI', 'VI', 'shear_stress']

# varnames = ['UI', 'VI']

varnames = ['oceanspeed']

varnames = ['shear', 'hi']



views = ['Aug_crash', 'May_crash', 'tripole_bug', 'Arctic_closeup', 'Arctic', 'Antarctic', 'EAC', 'Agulhas', 'Kuroshio', 'Gulf_Stream', 'ACC']

views = ['tripole_bug', 'Arctic_closeup']

views = ['EAC', 'Agulhas', 'Kuroshio', 'Gulf_Stream', 'ACC']

views = ['Arctic', 'Antarctic']

# views = ['ACC']

for varname in tqdm_notebook(varnames, leave=False, desc='variables'):
    fnum = 0  # enumerate doesn't work with tqdm_notebook...
    for fpath in tqdm_notebook(icehFileList, leave=False, desc='day'):
#             print(fpath)
        fnum += fnum + 1
        fname = os.path.basename(fpath)
        try:
            date = fname.split('.')[1]
            month = int(date.split('-')[1])
        except:
            date = str(fnum)
            month = ''
            
        viewstodo = []
        outnames = []
        for view in views:
            outname = expt + '_' + view + '_' + varname + '_' + date + '.png'
            if os.path.exists(os.path.join(figdir, outname)):
                print('Skipping ' + outname + ' (file exists)')
            else:
                viewstodo.append(view)
                outnames.append(outname)
        if len(viewstodo)>0:
            try:
                del var
            except:
                pass
            try:
                del varmask
            except:
                pass
            ncFile = Dataset(fpath, mode='r')
            if varname.endswith('speed'):
                if varname == 'icespeed':
                    try:
                        u = ncFile.variables['uvel'][:][0,:,:]
                        v = ncFile.variables['vvel'][:][0,:,:]
                    except:
                        u = ncFile.variables['uvel_m'][:][0,:,:]
                        v = ncFile.variables['vvel_m'][:][0,:,:]
                elif varname == 'oceanspeed':
                    try:
                        u = ncFile.variables['uocn'][:][0,:,:]
                        v = ncFile.variables['vocn'][:][0,:,:]
                    except:
                        u = ncFile.variables['uocn_m'][:][0,:,:]
                        v = ncFile.variables['vocn_m'][:][0,:,:]
                else:
                    print('Error: ' + varname + ' unknown')
                    raise 
                var = np.ma.sqrt(u**2 + v**2)  # u,v are co-located on B grid
#                 elif varname=='shear_stress':
#                     # NB: this is the maximum shearing stress, not the shearing strain
#                     # https://en.wikipedia.org/wiki/Cauchy_stress_tensor#Maximum_and_minimum_shear_stresses
#                     sigi  = ncFile.variables['SIGI' ][:][0,:,:]
#                     sigii = ncFile.variables['SIGII'][:][0,:,:]
#                     var = abs(sigi-sigii)/2 # BUG: INCORRECT! SIGI, SIGII are invariants, not stresses - see notebook p588
            else:
                try:
                    var = ncFile.variables[varname][:][0,:,:]
                except:
                    var = ncFile.variables[varname + '_m'][:][0,:,:]
            ncFile.close()

            varmask = np.ma.getmask(var)
            if varname.startswith('shear'):
                var = np.where(var>0, np.log10(var), -2)

#             varmask = np.ma.getmask(var)
#             np.ma.masked_where(varmask,var)

            var = np.ma.masked_where(varmask,var)


       
            vnum = -1  # enumerate doesn't work with tqdm_notebook...
            for view in tqdm_notebook(viewstodo, leave=False, desc='views'):
                vnum += 1
                outname = outnames[vnum]
#                 print('doing ' + outname)
                fact = 1  # scaling factor for var
                parlabels = [False,False,False,False]

                if varname == 'aice':
                    desc = 'Ice concentration'
                    levels = np.arange(0,1.01,.01)
                    ticks = [0,.2,.4,.6,.8,1]
                    ext = 'neither'
                    cmp = cm.cm.ice
                elif varname == 'hi':
                    desc = 'Ice thickness (m)'
                    levels = np.arange(0,6.01,.06)
                    ticks = range(7)
                    ext = 'max'
                    cmp = cm.cm.ice
                elif varname in ['uvel', 'UI']:
                    desc = 'Ice x velocity (m/s)'
                    levels = np.arange(-1,1.01,.01)
                    ticks = np.arange(levels[0],levels[-1]+.001,0.2)
                    ext = 'both'
                    cmp = cm.cm.balance
                elif varname in ['vvel', 'VI']:
                    desc = 'Ice y velocity (m/s)'
                    levels = np.arange(-1,1.01,.01)
                    ticks = np.arange(levels[0],levels[-1]+.001,0.2)
                    ext = 'both'
                    cmp = cm.cm.balance
                elif varname == 'uocn':
                    desc = 'Ocean x velocity (m/s)'
                    levels = np.arange(-1,1.01,.01)
    #                     levels = np.arange(-.25,.2501,.01)
                    ticks = np.arange(levels[0],levels[-1]+.001,0.2)
                    ext = 'both'
                    cmp = cm.cm.balance
                elif varname == 'vocn':
                    desc = 'Ocean v velocity (m/s)'
                    levels = np.arange(-1,1.01,.01)
                    ticks = np.arange(levels[0],levels[-1]+.001,0.2)
                    ext = 'both'
                    cmp = cm.cm.balance
                elif varname == 'ssu_i':
                    desc = 'ssu_i (m/s)'
                    levels = np.arange(-1,1.01,.01)
                    ticks = np.arange(levels[0],levels[-1]+.001,0.2)
                    ext = 'both'
                    cmp = cm.cm.balance
                elif varname == 'ssv_i':
                    desc = 'ssv_i (m/s)'
                    levels = np.arange(-1,1.01,.01)
                    ticks = np.arange(levels[0],levels[-1]+.001,0.2)
                    ext = 'both'
                    cmp = cm.cm.balance
                elif varname == 'icespeed':
                    desc = 'Ice speed (m/s)'
                    levels = np.arange(0,1.01,.01)
                    ticks = np.arange(levels[0],levels[-1]+.001,0.2)
                    ext = 'max'
                    cmp = cm.cm.thermal
                elif varname == 'oceanspeed':
                    desc = 'Ocean surface speed (m/s)'
                    levels = np.arange(0,1.01,.01)
                    ticks = np.arange(levels[0],levels[-1]+.001,0.2)
                    ext = 'max'
                    cmp = cm.cm.thermal
                elif varname == 'divu':
                    desc = 'Ice divergence rate (%/day)'
                    levels = np.arange(-1000,1000.01,20)
                    ticks = np.rint(np.arange(levels[0],levels[-1]+.001,250.0))
    #                     var = np.copysign(np.log10(np.abs(var)), var)
    #                     desc = 'Log of ice divergence rate (%/day)'
    #                     levels = np.arange(-3,3.01,.05)
    #                     ticks = np.rint(np.arange(levels[0],levels[-1],1.0))
                    ext = 'both'
                    cmp = cm.cm.balance
                elif varname.startswith('shear'):
    # #                     desc = 'Ice strain rate (shear) (%/day)'
    # #                     levels = np.arange(0,300.01,10)
    # #                     ticks = np.rint(np.arange(levels[0],levels[-1],30.0))
    # #                     ext = 'max'
    # #                     noice = (var <= 0).astype('int')
    #                     varcopy = var
    # # #                     var = np.copysign(np.log10(np.abs(var)), var)
    # #                     var = np.log10(var)
    # #                     var = np.where(varcopy<=0, -2, var)
    # #                     var = np.where(varcopy==np.nan, np.nan, var)
#                     if vnum==0:
#                         var = np.where(var>0, np.log10(var), -2)
#     #                     var = np.ma.masked_where(np.ma.getmask(varcopy),var)
                    desc = 'Log of ice shear rate (%/day)'
                    levels = np.arange(-2,3.01,.05)
                    ticks = np.rint(np.arange(levels[0],levels[-1]+.001,1.0))
                    ext = 'both'
                    cmp = cm.cm.thermal
                elif varname in ['strairx', 'tau_x']:
                    desc = 'Wind stress, x direction (Pa)'
                    levels = np.arange(-0.1,0.101,.002)
                    ticks = np.arange(levels[0],levels[-1]+.001,0.1)
                    ext = 'both'
                    cmp = cm.cm.balance
                elif varname in ['strairy', 'tau_y']:
                    desc = 'Wind stress, y direction (Pa)'
                    levels = np.arange(-0.1,0.101,.002)
                    ticks = np.arange(levels[0],levels[-1]+.001,0.1)
                    ext = 'both'
                    cmp = cm.cm.balance
                elif varname == 'strocnx':
                    desc = '- Ocean-ice stress, x direction (Pa)'
                    levels = np.arange(-0.1,0.101,.002)
                    ticks = np.arange(levels[0],levels[-1]+.001,0.1)
                    ext = 'both'
                    cmp = cm.cm.balance
                    fact = -1
                elif varname == 'strocny':
                    desc = '- Ocean-ice stress, y direction (Pa)'
                    levels = np.arange(-0.1,0.101,.002)
                    ticks = np.arange(levels[0],levels[-1]+.001,0.1)
                    ext = 'both'
                    cmp = cm.cm.balance
                    fact = -1
                elif varname == 'sst':
                    desc = 'SST (C)'
                    levels = np.arange(-3,10.1,.25)
    #                     ticks = np.arange(levels[0],levels[-1]+.001,2)
                    ticks = np.arange(-4,100,2)
                    ext = 'both'
    #                     cmp = plt.cm.gist_ncar
                    cmp = cm.cm.thermal
                elif varname.startswith('SIG'):
                    desc = 'Stress invariant ' + varname + ' (units??)'
                    levels = np.linspace(-1e6,1e6,50)
                    ticks = np.arange(levels[0],levels[-1]+1e4,1e5)
                    ext = 'both'
                    cmp = cm.cm.balance
                else:
                    desc = '***UNKNOWN***'
                    levels = np.arange(0,1.01,.1)
                    ticks = [0,.2,.4,.6,.8,1]
                    ext = 'neither'
                    cmp = cm.cm.balance



#                 var = 
#                 np.ma.masked_where(varmask,var)


                font = {'size':24}
                tick_font=24

                fig = plt.figure(figsize=(30,30))
                if view == 'Arctic':
                    m = Basemap(projection ='npstere',boundinglat=45,lon_0=-10,resolution='l')
                elif view == 'Arctic_closeup':
                    m = Basemap(projection ='npstere',boundinglat=70,lon_0=-10,resolution='l')
                elif view == 'Aug_crash':
                    m = Basemap(projection='stere',llcrnrlat=67,urcrnrlat=75,
                                llcrnrlon=-110,urcrnrlon=-90,lat_0=75,lon_0=-100,resolution='l')
                elif view == 'May_crash':
                    m = Basemap(projection='stere',llcrnrlat=66,urcrnrlat=85,
                                llcrnrlon=70,urcrnrlon=110,lat_0=75,lon_0=80,resolution='l')
                elif view == 'tripole_bug':
                    m = Basemap(projection='stere',llcrnrlat=72,urcrnrlat=85,
                                llcrnrlon=70,urcrnrlon=110,lat_0=75,lon_0=80,resolution='l')
                elif view == 'EAC':
                    m = Basemap(projection='cyl',
                                llcrnrlon=142-360,urcrnrlon=175-360,
                                llcrnrlat=-48,urcrnrlat=-20,
                                resolution='l')
                    parlabels = [True,False,False,True]
                elif view == 'Agulhas':
                    m = Basemap(projection='cyl',
                                llcrnrlon=0,urcrnrlon=50,
                                llcrnrlat=-48,urcrnrlat=-15,
                                resolution='l')
                    parlabels = [True,False,False,True]
                elif view == 'Kuroshio':
                    m = Basemap(projection='cyl',
                                llcrnrlon=125-360,urcrnrlon=165-360,
                                llcrnrlat=25,urcrnrlat=50,
                                resolution='l')
                    parlabels = [True,False,False,True]
                elif view == 'Gulf_Stream':
                    m = Basemap(projection='cyl',
                                llcrnrlon=-98,urcrnrlon=-50,
                                llcrnrlat=15,urcrnrlat=45,
                                resolution='l')
                    parlabels = [True,False,False,True]
                elif view == 'ACC':
                    m = Basemap(projection ='spstere',boundinglat=-30,lon_0=170,resolution='l')
                    parlabels = [False,False,False,False]
                else:
                    m = Basemap(projection ='spstere',boundinglat=-50,lon_0=170,resolution='l')

                # m = Basemap(projection ='ortho',resolution='l',lat_0=90,lon_0=45,
                #            llcrnrx=-m.urcrnrx/2.,llcrnry=-m.urcrnry/2.,urcrnrx=m.urcrnrx/2.,urcrnry=m.urcrnry/2.)
                # #             llcrnrlat=60,urcrnrlat=60,
                # #             llcrnrlon=-45,urcrnrlon=45)
                # #             llcrnrlon=loncorners[0],urcrnrlon=loncorners[2]) ,boundinglat=48
                x,y = m(*(lon_t,lat_t))
                ctr = m.contourf(x,y,fact*var,levels=levels,cmap=cmp, extend=ext) #, vmin=0, vmax=1.1)
    #                 ctr.cmap.set_under(color=cmp(np.min(levels)), alpha=None)
    #                 ctr.cmap.set_over(color=cmp(np.max(levels)), alpha=None)
                ctr.cmap.set_under(color=cmp(0), alpha=None)
                ctr.cmap.set_over(color=cmp(.999), alpha=None)
    #                 if view.startswith('Arctic'):
    #                     xobs,yobs = m(*(obs_lon_NH,obs_lat_NH))
    #                     m.contour(xobs,yobs,CN_obs_NH[month-1],[0.3],colors='r')
    #                 else:
    #                     xobs,yobs = m(*(obs_lon_SH,obs_lat_SH))
    #                     m.contour(xobs,yobs,CN_obs_SH[month-1],[0.3],colors='r')

                plt.title(desc + ', ' + date,font)
                cbar = m.colorbar(ctr, location = 'bottom', pad = "6%")
                cbar.set_label(desc,size=tick_font)
                cbar.set_ticks(ticks)
                cbar_labels=plt.getp(cbar.ax.axes,'xticklabels')
                plt.setp(cbar_labels,fontsize=tick_font)
                m.drawmapboundary(fill_color='gray') # background color - for non-ocean areas
                # fill continents, set lake color same as ocean color.
                # m.fillcontinents(color='gray',lake_color='gray')
                # draw parallels and meridians.
                # label parallels on right and top
                # meridians on bottom and left
                parallels = np.arange(-80.,81,10.)
                # labels = [left,right,top,bottom]
    #                 m.drawparallels(parallels,color='white')#,labels=[False,True,True,False],size=tick_font)
                m.drawparallels(parallels,color='white',labels=parlabels,size=tick_font)

    #                 meridians = np.arange(0.,351.,20.)
                meridians = np.arange(0.,351.,10.)
                m.drawmeridians(meridians,labels=[True,False,False,True],size=tick_font,color='white')
                savefigure(outname)
                plt.close()

HBox(children=(IntProgress(value=0, description='variables', max=2), HTML(value='')))

HBox(children=(IntProgress(value=0, description='day', max=9), HTML(value='')))

/g/data3/hh5/public/apps/miniconda3/envs/analysis3-18.01/lib/python3.6/site-packages/ipykernel/__main__.py:112: RuntimeWarning: divide by zero encountered in log10


HBox(children=(IntProgress(value=0, description='views', max=2), HTML(value='')))

/g/data3/hh5/public/apps/miniconda3/envs/analysis3-18.01/lib/python3.6/site-packages/mpl_toolkits/basemap/__init__.py:1708: MatplotlibDeprecationWarning: The axesPatch function was deprecated in version 2.1. Use Axes.patch instead.
  limb = ax.axesPatch
/g/data3/hh5/public/apps/miniconda3/envs/analysis3-18.01/lib/python3.6/site-packages/mpl_toolkits/basemap/__init__.py:1711: MatplotlibDeprecationWarning: The axesPatch function was deprecated in version 2.1. Use Axes.patch instead.
  if limb is not ax.axesPatch:


HBox(children=(IntProgress(value=0, description='views', max=2), HTML(value='')))

HBox(children=(IntProgress(value=0, description='views', max=2), HTML(value='')))